<a href="https://colab.research.google.com/github/zoya-sanjeev/News-Classification/blob/master/NewsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT LIBRARIES:


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from tensorflow import keras
layers = keras.layers
models = keras.models

LOAD DATASET

In [2]:
data=pd.read_csv('bbc-text.csv')
data.head()


,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


SPLITTING THE DATASET

In [0]:
x=data['text']
y=data['category']
x_train_text,x_test_text,y_train_cat,y_test_cat=train_test_split(x,y,test_size=0.2)
train_size = int(len(data) * .8)


DATA PREPARATION

In [0]:
max_words=1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)
tokenize.fit_on_texts(x_train_text)
x_train = tokenize.texts_to_matrix(x_train_text)
x_test = tokenize.texts_to_matrix(x_test_text)


In [0]:
encoder = LabelEncoder()
encoder.fit(y_train_cat)
y_train = encoder.transform(y_train_cat)
y_test = encoder.transform(y_test_cat)


In [0]:
#one hot encoding
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [11]:
#optional
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1780, 1000)
x_test shape: (445, 1000)
y_train shape: (1780, 5)
y_test shape: (445, 5)


MODEL

In [12]:
batch_size = 16
epochs = 3
drop_ratio = 0.5

model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


Epoch 1/3
101/101 [==============================] - 1s 7ms/step - loss: 0.3616 - accuracy: 0.8945 - val_loss: 0.1907 - val_accuracy: 0.9438
Epoch 2/3
101/101 [==============================] - 1s 6ms/step - loss: 0.0317 - accuracy: 0.9963 - val_loss: 0.1472 - val_accuracy: 0.9494
Epoch 3/3
101/101 [==============================] - 1s 6ms/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 0.1389 - val_accuracy: 0.9326


EVALUATING THE MODEL

In [13]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

28/28 [==============================] - 0s 2ms/step - loss: 0.1021 - accuracy: 0.9685
Test loss: 0.10214938968420029
Test accuracy: 0.968539297580719


TESTING THE MODEL

In [14]:
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(x_test_text.iloc[i][:50], "...")
    print('Actual label:' + y_test_cat.iloc[i])
    print("Predicted label: " + predicted_label + "\n")  



mcclaren hails boro s uefa spirit middlesbrough bo ...
Actual label:sport
Predicted label: sport

newry to fight cup exit in courts newry city are e ...
Actual label:sport
Predicted label: sport

deutsche attacks yukos case german investment bank ...
Actual label:business
Predicted label: business

rapper films music video in jail a us rapper await ...
Actual label:entertainment
Predicted label: entertainment

uk youth  interested  in politics the majority of  ...
Actual label:politics
Predicted label: politics

bortolami predicts dour contest italy skipper marc ...
Actual label:sport
Predicted label: sport

us bank in $515m sec settlement five bank of ameri ...
Actual label:business
Predicted label: business

collins named uk athletics chief uk athletics has  ...
Actual label:sport
Predicted label: sport

nasdaq planning $100m-share sale the owner of the  ...
Actual label:business
Predicted label: business

assembly ballot papers  missing  hundreds of ballo ...
Actual label:politics
P